In [1]:
import pandas as pd
import numpy as np
import os
import sys
from collections import OrderedDict

In [2]:
def errorMessage(setActual, valor): 
    print('ERROR')
    print('Se ha detectado un valor inexistente en el Catalogo', setActual)
    print('El valor no registrado es: ', valor)
    print('Favor de agregar el valor faltante a el catalago y volver a intentar')
    print('Ingrese ENTER para continuar')
    print('Ingrese cualquier otra tecla para salir')
    aux = input()
    if not aux:
        return True
    else: 
        sys.exit()

        
def setDataFileType(typ):
            typ = typ.Upper()
            if typ.find("SIX") != -1:
                return "SIX"
            elif typ.find("SEVEN") != -1:
                return "SEVEN ELEVEN"
            elif typ.find("ADF") != -1:
                return "ADF & 3B"
            elif typ.find("3B") != -1:
                return "ADF & 3B"
            else:
                return "OXXO"

def fixDateValues(monthyValues):
    year = []
    column = monthyValues.columns.tolist()
    for month in column:
        year.append(month[-2:])
    res = list(OrderedDict.fromkeys(year))
    return res
    

In [70]:
## Definir el tipo de base de dato a procesar, encuentra que tipo de archivo es en base al input.
## El input SIEMPRE tendra el nombre del archivo
def setDataFileType(func):
    def _setDataFileType(*args, **kw):
        ## Cargar los datos
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1].upper()
        
        ## Encuentra el tipo de archivo y regresa los valores
        if typ.find("SIX") != -1:
            return db, "SIX"
        elif typ.find("SEVEN") != -1:
            return db, "SEVEN ELEVEN"
        elif typ.find("ADF") != -1:
            return db, "ADF & 3B"
        elif typ.find("3B") != -1:
            return db, "ADF & 3B"
        else:
            return db, "OXXO"
    return _setDataFileType

## Preparar la base para la homologación, esto se le hace a todas menos ADF y 3B. 
## Preparar las bases reduce la cantidad de lineas a leer, esto hace el programa más eficiente.     
def databaseSetUP(func):
    def _databaseSetUP(*args, **kw):
        ## Cargar los datos
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1].upper()
        setActual = "Set Up"
        valor = " "
        
        ## Comienza el loop
        flag = True
        while flag:
            ## Revisa que tipo de base es y las maneja
            if typ == "ADF & 3B":
                return db, typ
            elif typ == "SEVEN ELEVEN":
                firstRun = True
                for index, row in db.iterrows():
                    key = row['Plaza.1'] + str(row['COD BARRAS']) + str(row['año'])
                    if firstRun:
                        res = list(OrderedDict.fromkeys(db.mes))
                        ez = row['Retornabilidad'] + str(row['Envase']) + str(row['Mililitros Unidad'])
                        data = [row['año'], row['Plaza.1'], row['Marca'], row['Empaque'], ez, key, row['Proveedor']]
                        name = ['Year', 'GZ', 'Line Extension', 'Multipack', 'EZ', 'key', 'Proveedor']
                        for value in res:
                            data.append(0)
                            name.append(value)
                        df = pd.DataFrame(data, name).T
                        table = df
                        firstRun = False
                    try:
                        month = row['mes']
                        val = table['key'][table['key'] == key].index[0]
                        table.loc[table.index[val], month] = row['HTLS']
                    except:
                        ez = row['Retornabilidad'] + str(row['Envase']) + str(row['Mililitros Unidad'])
                        data = [row['año'], row['Plaza.1'], row['Marca'], row['Empaque'], ez, key, row['Proveedor']]
                        name = ['Year', 'GZ', 'Line Extension', 'Multipack', 'EZ', 'key', 'Proveedor']
                        for value in res:
                            data.append(0)
                            name.append(value)
                        df = pd.DataFrame(data, name).T
                        table = pd.concat([table, df], axis = 0, ignore_index=True)
                        month = row['mes']
                        val = table['key'][table['key'] == key].index[0]
                        table.loc[table.index[val], month] = row['HTLS']
                flag = False
            else:
                firstRun = True
                for index, row in db.iterrows():
                    key = row['OfV Description'] + str(row['SKU'])
                    if firstRun:
                        res = list(OrderedDict.fromkeys(db.Month))
                        ez = row['Returnability'] + row['Primary Pack Type'] + row['Primary Pack Group'] + row['HMex Pack Group'] + row['Pack Size']
                        data = [row['Year'], row['GZ Split TVN-VDM'], row['Line Extension'], row['Multipack'], ez, key]
                        name = ['Year', 'GZ', 'Line Extension', 'Multipack', 'EZ', 'key']
                        for value in res:
                            data.append(0)
                            name.append(value)
                        df = pd.DataFrame(data, name).T
                        table = df
                        firstRun = False
                    try:
                        month = row['Month']
                        val = table['key'][table['key'] == key].index[0]
                        table.loc[table.index[val], month] = row['Value']
                    except:
                        ez = row['Returnability'] + row['Primary Pack Type'] + row['Primary Pack Group'] + row['HMex Pack Group'] + row['Pack Size']
                        data = [row['Year'], row['GZ Split TVN-VDM'], row['Line Extension'], row['Multipack'], ez, key]
                        name = ['Year', 'GZ', 'Line Extension', 'Multipack', 'EZ', 'key']
                        for value in res:
                            data.append(0)
                            name.append(value)
                        df = pd.DataFrame(data, name).T
                        table = pd.concat([table, df], axis = 0, ignore_index=True)
                        month = row['Month']
                        val = table['key'][table['key'] == key].index[0]
                        table.loc[table.index[val], month] = row['Value']
                flag = False
        return table, typ
    return _databaseSetUP

## Obtiene el canal de cada base, porque cada base tiene un canal predeterminado no ocupamos un catalogo
def setCanal(func):
    def _setCanal(*args, **kw):
        ## Cargan los valores
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1]
        tablaFinal = [None] * 34
        setActual = "Canal"
        valor = "Catalogo Inexistente / Incorrecto"
        canal = []
        canalInt = []
        canalMa = []
        length = len(db)
        flag = True
        ## Comienza el loop
        while flag:
            if typ == "SIX":
                try:
                    for x in range(length):
                        valor = x
                        canal.append('SIX')
                        canalInt.append('OFF TRAD')
                        canalMa.append('SIX / MODELORAMA')
                    flag = False
                except:
                    flag = errorMessage(setActual, valor)      
            elif typ == "SEVEN ELEVEN":
                try:
                    for x in range(length):
                        valor = x
                        canal.append('7 Eleven')
                        canalInt.append('Off Modern')
                        canalMa.append('OTHER -CS + PROX')
                    flag = False
                except:
                    flag = errorMessage(setActual, valor)
                    
            elif typ == "ADF & 3B":
                try:
                    for x in range(length):
                        valor = x
                        canal.append(db['Cliente Cadena'][x])
                        canalInt.append('Off Modern')
                        canalMa.append('OTHER -CS + PROX')
                    flag = False
                except:
                    flag = errorMessage(setActual, valor)
            else:
                try:
                    for x in range(length):
                        valor = x
                        canal.append('SIX')
                        canalInt.append('OFF TRAD')
                        canalMa.append('SIX / MODELORAMA')
                    flag = False
                except:
                    flag = errorMessage(setActual, valor)

            tablaFinal[0] = pd.Series(canal)
            tablaFinal[1] = pd.Series(canalInt)
            tablaFinal[2] = pd.Series(canalMa)
        return db, typ, tablaFinal
    return _setCanal

def setUE(func):
    def _setUE(*args, **kw):
        ## Cargan los valores
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1]
        tablaFinal = func(*args, **kw)[2]
        setActual = "Canal"
        valor = " "
        unidadesDeEmpaque = []
        flag = True
        ## Comienza el loop
        while flag:
            
            if typ == "ADF & 3B":
                try:
                    emCat = pd.read_excel('../Catalogs/CatalogoADFy3B.xlsx', sheet_name = 'Pack')
                    multipacks = db.Multipacks
                    for value in multipacks:
                        loc = emCat.loc[emCat.Multipacks == value]
                        unidadesDeEmpaque.append(loc['Val'].iloc[0]) 
                    flag = False
                except:
                    flag = errorMessage(setActual, valor)
            elif typ == "SEVEN ELEVEN":
                try:
                    unidadesDeEmpaque = db.Multipack
                    flag = False
                except:
                    flag = errorMessage(setActual, valor)
            else:
                try:
                    multipacks = db.Multipack
                    for value in multipacks:
                        valor = value
                        loc = [int(s) for s in value.split() if s.isdigit()]     
                        unidadesDeEmpaque.append(loc[0])
                    flag = False
                    unidadesDeEmpaque = pd.Series(unidadesDeEmpaque) 
                except:
                    flag = errorMessage(setActual, valor)
            tablaFinal[3] = pd.Series(unidadesDeEmpaque)
        return db, typ, tablaFinal
    return _setUE

def setGZ(func):
    def _setGZ(*args, **kw):
        ## Se cargan los valores
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1]
        tablaFinal = func(*args, **kw)[2]
        gz = db["GZ"]
        setActual = "GZ"
        valor = "-1"
        region = []
        gz1 = []
        flag = True
        ## Comienza el loop
        while flag:
            try:
                
                if typ == "ADF & 3B":
                    gzCat = pd.read_excel('../Catalogs/CatalogoADFy3B.xlsx', sheet_name = 'Gz')
                elif typ == "SEVEN ELEVEN":
                    gzCat = pd.read_excel('../Catalogs/CatalogoSevenEleven.xlsx', sheet_name = 'Gz')
                elif typ == "OXXO":
                    gzCat = pd.read_excel('../Catalogs/CatalogoOXXO.xlsx', sheet_name = 'Gz')
                else:
                    gzCat = pd.read_excel('../Catalogs/CatalogoSIX.xlsx', sheet_name = 'Gz')
                    
                for value in gz:
                    valor = value
                    loc = gzCat.loc[gzCat.Plaza == value]  
                    region.append(loc['REGION'].iloc[0])
                    gz1.append(loc['GZ'].iloc[0])
                
                flag = False
            except:
                flag = errorMessage(setActual, valor)

            tablaFinal[4] = pd.Series(region)
            tablaFinal[5] = pd.Series(gz1)
        return db, typ, tablaFinal    
    return _setGZ

def setMarca(func):
    def _setMarca(*args, **kw):
        ## Se cargan los valores
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1]
        tablaFinal = func(*args, **kw)[2]
        setActual = "Marca"
        valor = " "
        marca_hom = []
        ext_marca_hom = []
        diamante_precio_est = []
        tipo_liq_est = []
        liq = []
        marca_hom2 = []
        ext_marca_hom2 = []
        diamante_precio_hom = []
        manu = []
        priceTierCMI = []
        categoryCMI = []
        alcoholSegmentsCMI = []
        liquidSegmentsCMI = []
        brandCondition = []
        topLineSOg = []
        secSG = []
        fabricante = []
        flag = True
        ## Comienza el loop
        while flag:
            
            le = db['Line Extension']
            try: 
                if typ == "ADF & 3B":
                    marcaCat = pd.read_excel('../Catalogs/CatalogoADFy3B.xlsx', sheet_name = 'Marca')
                elif typ == "SEVEN ELEVEN":
                    marcaCat = pd.read_excel('../Catalogs/CatalogoSevenEleven.xlsx', sheet_name = 'Marca')
                elif typ == "OXXO":
                    marcaCat = pd.read_excel('../Catalogs/CatalogoOXXO.xlsx', sheet_name = 'Marca')
                else:
                    marcaCat = pd.read_excel('../Catalogs/CatalogoSIX.xlsx', sheet_name = 'Marca')           
                for value in le:
                    valor = value
                    loc = marcaCat.loc[marcaCat['Line Extension'] == value]
                    marca_hom.append(loc['MARCA_HOM'].iloc[0])
                    ext_marca_hom.append(loc['EXT_MARCA_HOM'].iloc[0])
                    diamante_precio_est.append(loc['DIAMANTE_PRECIO_EST'].iloc[0])
                    tipo_liq_est.append(loc['TIPO_LIQ_EST'].iloc[0])
                    liq.append(loc['LIQ'].iloc[0])       
                    marca_hom2.append(loc['MARCA_HOM.1'].iloc[0])
                    ext_marca_hom2.append(loc['EXT_MARCA_HOM.1'].iloc[0])
                    diamante_precio_hom.append(loc['DIAMANTE_PRECIO_HOM'].iloc[0])
                    manu.append(loc['Manufacturer_CMI'].iloc[0])
                    if (loc['Manufacturer_CMI'].iloc[0]) == "Nosotros":
                        fabricante.append("PROPIA")
                    else:
                        fabricante.append("OTROS")
                    priceTierCMI.append(loc['Price_Tier_CMI'].iloc[0])
                    categoryCMI.append(loc['Category_CMI'].iloc[0])  
                    alcoholSegmentsCMI.append(loc['Alcohol_Segments_CMI'].iloc[0])
                    liquidSegmentsCMI.append(loc['Liquid_Segment_CMI'].iloc[0])
                    brandCondition.append(loc['BRAND_CONDITION'].iloc[0])
                    topLineSOg.append(loc['TOP LINE SOG (5 SOGS)'].iloc[0])
                    secSG.append(loc['Secondary_Source_of_Growth'].iloc[0])
                flag = False
            except:
                flag = errorMessage(setActual, valor)  
          
        tablaFinal[6] = pd.Series(marca_hom)
        tablaFinal[7] = pd.Series(ext_marca_hom)
        tablaFinal[8] = pd.Series(diamante_precio_est)
        tablaFinal[9] = pd.Series(tipo_liq_est)
        tablaFinal[10] = pd.Series(liq)
        tablaFinal[11] = pd.Series(marca_hom2)
        tablaFinal[12] = pd.Series(ext_marca_hom2)
        tablaFinal[13] = pd.Series(diamante_precio_hom)
        tablaFinal[14] = pd.Series(manu)
        tablaFinal[15] = pd.Series(priceTierCMI)
        tablaFinal[16] = pd.Series(categoryCMI)
        tablaFinal[17] = pd.Series(alcoholSegmentsCMI)
        tablaFinal[18] = pd.Series(liquidSegmentsCMI)
        tablaFinal[19] = pd.Series(brandCondition)
        tablaFinal[20] = pd.Series(topLineSOg)
        tablaFinal[21] = pd.Series(secSG)
        tablaFinal[22] = pd.Series(fabricante)
          
        return db, typ, tablaFinal   
    return _setMarca

def setReturnabiliy(func):
    def _setReturnability(*args, **kw):
        ## Se cargan los valores
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1]
        tablaFinal = func(*args, **kw)[2]
        setActual = "Returnability"
        valor = " "
        returnability = []
        packtypeHOM = []
        primaryPacktypeGroup = []
        groupPacksize = []
        packSizeHOM = []
        packtypeOk2 = []
        categorias = []
        packSize1Home = []
        pConcatenado1 = []
        pConcatenado2 = []
        presentacion = []
        flag = True
        ## Comienza el loop
        while flag:
            
            try: 
                if typ == "ADF & 3B":
                    returnabilityCat = pd.read_excel('../Catalogs/CatalogoADFy3B.xlsx', sheet_name = 'Returnability')
                    ez = db['Presentación CMM']
                elif typ == "SEVEN ELEVEN":
                    returnabilityCat = pd.read_excel('../Catalogs/CatalogoSevenEleven.xlsx', sheet_name = 'Returnability')
                    ez = db['EZ']
                elif typ == "OXXO":
                    returnabilityCat = pd.read_excel('../Catalogs/CatalogoOXXO.xlsx', sheet_name = 'Returnability')
                    ez = db['EZ']
                else:
                    returnabilityCat = pd.read_excel('../Catalogs/CatalogoSIX.xlsx', sheet_name = 'Returnability')
                    ez = db['EZ']
                
                for value in ez:   
                    valor = value
                    loc = returnabilityCat.loc[returnabilityCat['llave'] == value]
                    returnability.append(loc['Returnability'].iloc[0])
                    if loc['Returnability'].iloc[0] == 'Non Returnable':
                        presentacion.append('NO RETORNABLE')
                    else: 
                        presentacion.append('RETORNABLE')
                    packtypeHOM.append(loc['Packtype_HOM'].iloc[0])
                    primaryPacktypeGroup.append(loc['Primary_Packtype_Group'].iloc[0])
                    groupPacksize.append(loc['Group_Packsize'].iloc[0])
                    packSizeHOM.append(loc['Packsize_HOM'].iloc[0])
                    packtypeOk2.append(loc['PACKTYPE_OK_2'].iloc[0])
                    categorias.append(loc['Categorias'].iloc[0])
                    packSize1Home.append(loc['Packsize1_HOME'].iloc[0])
                    pConcatenado1.append(loc['Packsize_concatenado1'].iloc[0])
                    pConcatenado2.append(loc['Packsize_concatenado2'].iloc[0])    
                flag = False
            except:
                flag = errorMessage(setActual, valor)

        tablaFinal[23] = pd.Series(returnability)
        tablaFinal[24] = pd.Series(packtypeHOM)
        tablaFinal[25] = pd.Series(primaryPacktypeGroup)
        tablaFinal[26] = pd.Series(groupPacksize)
        tablaFinal[27] = pd.Series(packSizeHOM)
        tablaFinal[28] = pd.Series(packtypeOk2)
        tablaFinal[29] = pd.Series(categorias)
        tablaFinal[30] = pd.Series(packSize1Home)
        tablaFinal[31] = pd.Series(pConcatenado1)
        tablaFinal[32] = pd.Series(pConcatenado2)
        tablaFinal[33] = pd.Series(presentacion) 
        return db, typ, tablaFinal 
    return _setReturnability

def generateTable(func):
    def _generateTable(*args, **kw):
        db = func(*args, **kw)[0]
        typ = func(*args, **kw)[1]
        valor = func(*args, **kw)[2]
        setActual = "Table Generation"
        values = " "
        flag = True
        while flag:
            
            if typ == "ADF & 3B":
                try:
                    añosPresentes = fixDateValues(db.iloc[:,9:])
                    size = len(db)
                    monthlyValues = db.iloc[:,9:].columns.tolist()
                    first = True
                    first2 = True
                    for value in añosPresentes:
                        años = []
                        for x in range(size):
                            años.append('20'+value)
                    años = pd.Series(años)
                    data = [años, valor[1], valor[0], valor[4], valor[5], valor[22], valor[6], valor[7], valor[8], valor[9], valor[10],  valor[33], valor[3], valor[2], valor[11], valor[12], valor[13], valor[23], valor[24], valor[25], valor[26], valor[27], valor[28], valor[29], valor[30], valor[31], valor[32], valor[14], valor[15], valor[16], valor[17], valor[18], valor[19], valor[20], valor[21]]
                    name = ['AÑO', 'CANAL INTEGRADO', 'CANAL', 'REGION', 'GZ', 'FABRICANTE', 'MARCA_EST', 'EXT_MARCA_EST', 'DIAMANTE_PRECIO_EST', 'TIPO_LIQ_EST', 'LIQ', 'PRESENTACION', 'UNIDADES_EMPAQUE', 'CANAL_MA', 'MARCA_HOM', 'EXT_MARCA_HOM', 'DIAMANTE_PRECIO_HOM', 'Retunability', 'Packtype_HOM', 'Primary_Packtype_Group', 'Group_Packsize', 'Packsize_HOM', 'PACKTYPE_OK_2', 'Categorias', 'Packsize1_HOME', 'Packsize_concatenado1', 'Packsize_concatenado2', 'Manufacturer_CMI', 'Price_Tier_CMI', 'Category_CMI', 'Alcohol_Segments_CMI', 'Liquid_Segment_CMI', 'BRAND_CONDITION', 'Primary_Source_of_Growth', 'Secondary_Source_of_Growth']
                    if first:
                        df = pd.DataFrame(data, name).T
                        table = df
                        first = False
                    else:
                        df = pd.DataFrame(data, name).T
                        table = pd.concat([table, df], axis = 0)
                    flag = False
                    df.to_excel("../" + typ + ".xlsx", index = False)
                except:
                    flag = errorMessage(setActual, values)
            else:
                try:
                    data = [db.Year, valor[1], valor[0], valor[4], valor[5], valor[22], valor[6], valor[7], valor[8], valor[9], valor[10],  valor[33], valor[3], valor[2], valor[11], valor[12], valor[13], valor[23], valor[24], valor[25], valor[26], valor[27], valor[28], valor[29], valor[30], valor[31], valor[32], valor[14], valor[15], valor[16], valor[17], valor[18], valor[19], valor[20], valor[21]]
                    name = ['AÑO', 'CANAL INTEGRADO', 'CANAL', 'REGION', 'GZ', 'FABRICANTE', 'MARCA_EST', 'EXT_MARCA_EST', 'DIAMANTE_PRECIO_EST', 'TIPO_LIQ_EST', 'LIQ', 'PRESENTACION', 'UNIDADES_EMPAQUE', 'CANAL_MA', 'MARCA_HOM', 'EXT_MARCA_HOM', 'DIAMANTE_PRECIO_HOM', 'Retunability', 'Packtype_HOM', 'Primary_Packtype_Group', 'Group_Packsize', 'Packsize_HOM', 'PACKTYPE_OK_2', 'Categorias', 'Packsize1_HOME', 'Packsize_concatenado1', 'Packsize_concatenado2', 'Manufacturer_CMI', 'Price_Tier_CMI', 'Category_CMI', 'Alcohol_Segments_CMI', 'Liquid_Segment_CMI', 'BRAND_CONDITION', 'Primary_Source_of_Growth', 'Secondary_Source_of_Growth']
                    g = db.iloc[:,7:]
                    monthlyValues = g.columns.tolist()
                    for month in monthlyValues:
                        data.append(db[month])
                        name.append(month)
                    df = pd.DataFrame(data, name).T
                    df.to_excel("../" + typ + ".xlsx", index = False)
                    flag = False
                except:
                    flag = errorMessage(setActual, values)
        return "La base de datos se compilo de manera exitosa"
    return _generateTable

class DatabaseGenerator:
    def __init__(self):
        self.db, self.typ = self.openDataFile()

    def openDataFile(self):
        entries = os.listdir("../Input/")
        i = 1
        for v in entries:
            print(str(i) + ". " + v)
            i = i + 1
        print("Seleccione el número del archivo: ")
        print("En caso de no tener el archivo en Input puede ingresar el directorio: ")
        aux = input()
        os.system("cls")
        if len(aux) <= 5:
            i = 1
            for v in entries:
                if aux == str(i):
                    return pd.read_excel("../Input/" + v), v
                i = i + 1
        else:
            return pd.read_excel(aux), v
    
    @generateTable
    @setReturnabiliy
    @setMarca
    @setGZ
    @setUE
    @setCanal
    @databaseSetUP
    @setDataFileType
    def generate(self):
        return self.db, self.typ

In [ ]:
decorated_add = DatabaseGenerator()
a = decorated_add.generate()

1. .DS_Store
2. InputADF&3B.xlsx
3. InputOxxo.xlsx
4. InputSevenEleven.xlsx
5. InputSIX.xlsx
Seleccione el número del archivo: 
En caso de no tener el archivo en Input puede ingresar el directorio: 
3
